In [ ]:
# duplicar as últimas amostras
# rodar o melhor modelo do tempo para predição de SST e firmeza e montar tabela

In [1]:
import cv2
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import math

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold

%load_ext autoreload

sys.path.append(os.getcwd()+'//..//scripts//')
path = os.getcwd() + '//..//'

from MNG import MNG
from MNGFeatures import MNGFeatures
from MNGFeaturesMeans import MNGFeaturesMeans
from MNGFeaturesSize import MNGFeaturesSize
from MNGModel import MNGModel

In [15]:
data_path = os.getcwd()+'/../resampling/all_data_repeated.csv'
old_data = pd.read_csv(data_path, sep=',', index_col=0)

second_half_df = old_data.iloc[600:]
to_be_deleted = [616,538,436,590,401,583,239,263,285,317,670,634,467,249,287,210,492,119,607,442,706,
343,183,201,394,665,559,345,217,214,455,699,269,39,92,689,279,504,1,272,97,28,277,48,462,137,313,686,575,291,
496,673,487,311,589,315,50,171,105,247,181,190,426,178,546,235,540,551,157,26,281,156,206,658,399,257,698,402,
290,350,20,533,330,479,111,128,332,107,264,124,574,195,294,639,672,208,186,597,227,611,457,267,324,163,295,653,
702,682,130,408,640,268,96,8,386,705,471,429,405,387]
rows_to_delete = second_half_df.iloc[to_be_deleted]

data = old_data.drop(rows_to_delete.index.values)
data = data.drop(columns=data.columns.values[:1710])

In [16]:
n_trees = 500
kf = KFold(n_splits=5, shuffle=True)

In [ ]:
# data = pd.read_csv(path+'features//size_all_half.csv', sep=';', index_col=0)
# palmer_sst = pd.read_csv(path+'//reference_values//primeiros_dados_palmer_sst.csv', sep=';', index_col=0)
# palmer_firm = pd.read_csv(path+'//reference_values//primeiros_dados_palmer_firmeza.csv', sep=';', index_col=0)

# data['sst'] = palmer_sst.values
# data['firmeza'] = palmer_firm.values

In [17]:
X = data.drop(columns=['time', 'sst', 'firmeza'])
Y_sst = data['sst']
Y_firm = data['firmeza']
Y_time = data['time']

predictions = pd.DataFrame(index=data.index.values, columns=['sst_predict', 'firmeza_predict', 'time_predict'])
real = pd.DataFrame(index=data.index.values, columns=['sst_real', 'firmeza_real', 'time_real'])

In [31]:
# # execute esta célula só se for fazer análise no arquivo com todas as variáveis
# data = data.drop(columns=data.columns.values[179:])

# # adicionando novamente as médias nas 5 regiões, porque no arquivo features_all há a substituição das médias com n = 20
# data_to_add = pd.read_csv(path+'features\\rgb_hsv_lab_n_regions_all_half.csv', sep=';', index_col=0)
# data[data_to_add.columns.values] = data_to_add[data_to_add.columns.values]

In [7]:
# apenas para repetição das 60 amostras
# index = np.append(data.index.values, [str(num) for num in np.arange(60)])

# predictions = pd.DataFrame(index=index, columns=['sst_predict', 'firmeza_predict', 'time_predict'])
# real = pd.DataFrame(index=index, columns=['sst_real', 'firmeza_real', 'time_real'])

# new_data = pd.DataFrame(data.iloc[-60:].values, index=np.arange(1,61),columns=data.columns)
# data = pd.concat([data,new_data])

In [18]:
i=1
for train_i, test_i in kf.split(X):
    print(i)
    X_train, X_test = X.iloc[train_i], X.iloc[test_i]
    Y_sst_train, __ = Y_sst.iloc[train_i], Y_sst.iloc[test_i]
    Y_firm_train, __ = Y_firm.iloc[train_i], Y_firm.iloc[test_i]
    Y_time_train, __ = Y_time.iloc[train_i], Y_time.iloc[test_i]
    
    rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_sst_train)
    Y_predicted = rf_model.predict(X_test)
    predictions['sst_predict'].iloc[test_i] = Y_predicted
    
    rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_firm_train)
    Y_predicted = rf_model.predict(X_test)
    predictions['firmeza_predict'].iloc[test_i] = Y_predicted
    
    rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_time_train)
    Y_predicted = rf_model.predict(X_test)
    predictions['time_predict'].iloc[test_i] = Y_predicted
    i=i+1

1
2
3
4
5


In [19]:
real['sst_real'] = data['sst']
real['firmeza_real'] = data['firmeza']
real['time_real'] = data['time']

# real.iloc[660:] = real.iloc[-120:-60].values
real.to_csv('real_repeated.csv', sep=',')

In [20]:
predictions.to_csv('dados_predict.csv', sep=';')
predictions['sst_real'] = real['sst_real']
predictions['firmeza_real'] = real['firmeza_real']
predictions['time_real'] = data['time']

# predictions.to_csv('dados_predict_reais.csv', sep=';')

In [21]:
def calc_tabela(data, att, range_max, range_min, intervals, consider_end):
    indexes = []
    bins = math.floor((range_max-range_min)/intervals)
    tabela = pd.DataFrame(data=0,index=np.arange(bins),columns=data.columns)
    
    for bin_num in range(bins):
        min_v = range_min if bin_num == 0 else range_min + intervals*bin_num + 1
        if bin_num!=(bins-1):
            max_v = min_v + intervals if bin_num == 0 else min_v + intervals - 1
        else:
            max_v = range_max
        indexes.append(str(min_v)+'-'+str(max_v))
        tabela.iloc[bin_num] = data[(min_v<=data[att]) & (data[att]<=max_v)].mean()
    
    tabela.index = indexes
    return tabela

In [22]:
range_max = 170
range_min = 20
intervals = 15

tabela_real = calc_tabela(real, 'time_real', range_max, range_min, intervals, False)
tabela_pred = calc_tabela(predictions, 'time_predict', range_max, range_min, intervals, False)

In [28]:
range_max = 190
range_min = 171
intervals = 9

tabela_real = calc_tabela(real, 'time_real', range_max, range_min, intervals, True)
tabela_pred = calc_tabela(predictions, 'time_predict', range_max, range_min, intervals, True)

In [32]:
tabela_real

,sst_real,firmeza_real,time_real
171-180,12.824,28.732333,180.0
181-190,14.950,6.035500,190.0
